# INST326 — Week 7 Exercises: Error Handling & Testing (Library Management)

**Focus (Week 7 only):** exception handling (`try` / `except` / `else` / `finally`), raising exceptions, defining simple custom exceptions, and **basic** unit testing with `unittest` (no fixtures beyond `setUp`/`tearDown`).

**Out of scope:** Anything introduced in Week 8 or later (e.g., inheritance, abstract classes, polymorphism, advanced testing/CI, design patterns).

> Context: Use a simple Library Management domain—books, members, catalog, and loans—to complete the tasks.


### Starter Helpers (Optional)

You may use or modify the minimal scaffolding below in your solutions. It intentionally avoids Week 8+ concepts.


In [ ]:
# Minimal, Week-7-safe scaffolding (no inheritance/ABCs).
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional, Dict, List
import json

class LibraryError(Exception):
    """Base library-related error for Week 7."""

class DuplicateBookError(LibraryError):
    pass

class OverdueLoanError(LibraryError):
    pass

@dataclass
class Book:
    isbn: str
    title: str
    copies: int = 1

@dataclass
class Member:
    member_id: str
    email: str

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def check_overdue(self) -> bool:
        if datetime.now() > self.due_date and not self.returned:
            return True
        return False

class Catalog:
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN already exists: {book.isbn}")
        if book.copies < 0:
            raise ValueError("copies must be non-negative")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def load_from_json(self, path: str) -> int:
        # Intentionally minimal; implement robust handling in exercises.
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        count = 0
        for item in data:
            self.add_book(Book(isbn=item["isbn"], title=item["title"], copies=item.get("copies", 1)))
            count += 1
        return count


## 1) Validate ISBN with exceptions

Write a function `validate_isbn(isbn: str) -> str` that:
- Strips hyphens/spaces
- Verifies it is 10 or 13 digits (numeric only after stripping)
- Raises `ValueError` with a helpful message when invalid
Return the normalized ISBN string when valid.

In [ ]:
# Your code here
def validate_isbn(isbn: str) -> str:
    normalized_isbn = isbn.replace('-', '').replace(' ', '')
    length = len(normalized_isbn)
    if length not in (10, 13):
        raise ValueError(
            f"Invalid ISBN length: Must be 10 or 13 digits after stripping. Found length {length} (Normalized: '{normalized_isbn}')."
        )
    if not normalized_isbn.isdigit():
        is_potential_isbn10_with_x = (
            length == 10 and normalized_isbn[-1].upper() == 'X'
        )
        
        error_message = f"Invalid ISBN format: Must contain only digits (0-9) after stripping. Found '{normalized_isbn}'."
        
        if is_potential_isbn10_with_x:
             error_message += "\n(Note: This ISBN looks like a valid ISBN-10 but contains the check digit 'X', which violates the 'numeric only' requirement.)"

        raise ValueError(error_message)    
    return normalized_isbn

# Example:
# print(validate_isbn("978-1-4028-9462-6"))  # -> '9781402894626'

## 2) Safe integer input for copies

Implement `parse_copies(text: str) -> int` that parses a positive integer for number of copies.
Use `try/except` to catch `ValueError`, and raise your own `ValueError` with a user-friendly message.

In [ ]:
# Your code here
def parse_copies(text: str) -> int:
    if not text:
        raise ValueError("Input cannot be empty. Please enter a number for the copies.")

    try:
        copies = int(text)
    except ValueError:
        raise ValueError(f"Invalid input: '{text}' is not a valid whole number.")

    if copies <= 0:
        raise ValueError(
            f"Invalid number of copies: Must be a positive integer greater than zero. Found {copies}."
        )

    return copies
    ...

# Examples:
# parse_copies("3") -> 3
# parse_copies("three") -> ValueError('copies must be a positive integer')

## 3) Custom exception: OverdueLoanError

Extend `Loan.check_overdue()` to **raise** `OverdueLoanError` if the book is overdue (instead of returning `True/False`). 
Catch this exception in a separate function `assert_not_overdue(loan: Loan)` that returns `True` when ok and `False` when overdue.

In [ ]:
# Your code here
@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def check_overdue(self) -> None:
        """Raise OverdueLoanError if the loan is overdue."""
        if datetime.now() > self.due_date and not self.returned:
            raise OverdueLoanError(
                f"Loan for ISBN {self.isbn} (member {self.member_id}) is overdue."
            )


def assert_not_overdue(loan: Loan) -> bool:
    try:
        loan.check_overdue()
        return True
    except OverdueLoanError:
        return False


# Tip: modify Loan.check_overdue or wrap/extend without Week 8+ concepts.

## 4) Robust date parsing

Write `parse_date(text: str) -> datetime` that accepts `YYYY-MM-DD`. If parsing fails, raise `ValueError('invalid date: ...')`.
Use `try/except` around `datetime.strptime`.

In [ ]:
# Your code here
from datetime import datetime

def parse_date(text: str) -> datetime:
    try:
        return datetime.strptime(text, "%Y-%m-%d")
    except ValueError:
        raise ValueError(f"invalid date: {text}")

# Example:
# parse_date("2025-10-27")

## 5) Late fee calculation with ZeroDivisionError guard

Define `per_day_late_fee(total_fee: float, days_late: int) -> float` that computes `total_fee / days_late`.
If `days_late` is 0, raise `ZeroDivisionError` with a clear message. Show a `try/except` usage example that prints a friendly message instead of crashing.

In [ ]:
# Your code here
def per_day_late_fee(total_fee: float, days_late: int) -> float:
   if days_late == 0:
        raise ZeroDivisionError("days_late cannot be zero when calculating late fee")
   return total_fee / days_late


# try/except demo here
try:
    fee = per_day_late_fee(10.0, 0)
    print(f"Per-day late fee: ${fee:.2f}")
except ZeroDivisionError as e:
    print(f"⚠️ Error: {e}")

## 6) Always-close file with try/finally

Implement `read_file_head(path: str, n: int=3) -> list[str]` that opens a UTF-8 text file and returns the first `n` lines (stripped).
Use explicit `try/finally` to ensure closing (even though `with` is preferred) to practice `finally` behavior.

In [ ]:
# Your code here
def read_file_head(path: str, n: int=3) -> list[str]:
    lines: list[str] = []
    f = None
    try:
        f = open(path, "r", encoding="utf-8")
        for _ in range(n):
            line = f.readline()
            if not line:
                break
            lines.append(line.strip())
    finally:
        if f is not None:
            f.close()
    return lines

## 7) Safe JSON catalog loading

Implement `safe_load_catalog(path: str, catalog: Catalog) -> int` that handles:
- `FileNotFoundError` → return 0
- `json.JSONDecodeError` → return 0
- `DuplicateBookError` → skip duplicate and continue
Return the number of **new** books added.

In [ ]:
# Your code here
import json

def safe_load_catalog(path: str, catalog: Catalog) -> int:
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return 0

    added = 0
    for item in data:
        try:
            catalog.add_book(Book(
                isbn=item["isbn"],
                title=item["title"],
                copies=item.get("copies", 1)
            ))
            added += 1
        except DuplicateBookError:
            # Skip duplicate and continue loading
            continue
    return added

## 8) Duplicate book detection

Write `add_unique_book(catalog: Catalog, book: Book)` that raises `DuplicateBookError` if the ISBN is already present, else adds it.
Demonstrate `try/except` around this call to log a brief message and continue.

In [ ]:
# Your code here
def add_unique_book(catalog: Catalog, book: Book) -> None:
    """Add a book only if its ISBN is not already in the catalog."""
    if catalog.get_book(book.isbn) is not None:
        raise DuplicateBookError(f"Book with ISBN {book.isbn} already exists.")
    catalog.add_book(book)


# --- Demo with try/except ---
catalog = Catalog()

books = [
    Book(isbn="111", title="Intro to Python"),
    Book(isbn="111", title="Duplicate Python Book"),
    Book(isbn="222", title="Data Science 101"),
]

for book in books:
    try:
        add_unique_book(catalog, book)
        print(f"Added book: {book.title} (ISBN: {book.isbn})")
    except DuplicateBookError as e:
        print(f"⚠️ Could not add book: {e}")

## 9) Timeout handling (simulated)

Create `fetch_cover_image(isbn: str, timeout_s: float=0.1) -> bytes` that **simulates** a timeout by raising `TimeoutError` when `timeout_s` < 0.05.
Write `get_cover_or_none(isbn)` that calls it in `try/except TimeoutError` and returns `None` on timeout.

In [ ]:
# Your code here
def fetch_cover_image(isbn: str, timeout_s: float=0.1) -> bytes:
    if timeout_s < 0.05:
        raise TimeoutError(f"Fetching cover for ISBN {isbn} timed out after {timeout_s}s.")
    return b"FAKE_IMAGE_DATA"

def get_cover_or_none(isbn: str):
    try:
        return fetch_cover_image(isbn)
    except TimeoutError as e:
        print(f"⚠️ Timeout while fetching cover: {e}")
        return None
    ...

## 10) Sanitizing member IDs

Implement `sanitize_member_id(value) -> str` that:
- Raises `TypeError` if not `str`
- Strips spaces, uppercases
- Raises `ValueError` if final form is empty or contains non-alphanumeric chars

In [ ]:
# Your code here
def sanitize_member_id(value) -> str:
    if not isinstance(value, str):
        raise TypeError("member_id must be a string")

    cleaned = value.strip().upper()

    if not cleaned:
        raise ValueError("member_id cannot be empty after stripping spaces")

    if not cleaned.isalnum():
        raise ValueError("member_id must contain only letters and digits")

    return cleaned

## 11) Convert asserts to exceptions

Given legacy code that uses `assert copies >= 0`, replace it with explicit `if` + `raise ValueError('copies must be non-negative')` in a function `validate_copies(copies: int) -> int` that returns the validated value.

In [ ]:
# Your code here
def validate_copies(copies: int) -> int:
    if copies < 0:
        raise ValueError("copies must be non-negative")
    return copies
    ...

## 12) Basic unit tests for `validate_isbn`

Create a `tests/`-style cell using `unittest` that verifies:
- Valid 10- and 13-digit ISBNs pass
- Bad inputs raise `ValueError`
Do **not** import any third-party packages.

In [ ]:
import unittest

def validate_isbn(isbn: str) -> str:
    """Validate that ISBN is a 10- or 13-digit numeric string (ignores dashes/spaces)."""
    if not isinstance(isbn, str):
        raise ValueError("ISBN must be a string")

    cleaned = isbn.replace("-", "").replace(" ", "")

    if len(cleaned) == 10 and cleaned.isdigit():
        return cleaned
    elif len(cleaned) == 13 and cleaned.isdigit():
        return cleaned
    else:
        raise ValueError("Invalid ISBN format or length")


class TestValidateISBN(unittest.TestCase):
    def test_valid_10_and_13(self):
        valid_isbns = [
            "0306406152",        # valid 10-digit
            "9780306406157",     # valid 13-digit
            "978-0-306-40615-7", # valid with dashes
            "0 306 40615 2"      # valid with spaces
        ]
        for isbn in valid_isbns:
            with self.subTest(isbn=isbn):
                result = validate_isbn(isbn)
                self.assertTrue(len(result) in (10, 13))
                self.assertTrue(result.isdigit())

    def test_invalid_values(self):
        invalid_isbns = [
            "123456789",         # Too short
            "12345678901234",    # Too long
            "ABCDEFGHIJ",        # Non-numeric
            "978-1-4028-9462-X", # Invalid character
            "",                  # Empty string
            1234567890           # Not a string
        ]
        for isbn in invalid_isbns:
            with self.subTest(isbn=isbn):
                with self.assertRaises(ValueError):
                    validate_isbn(isbn)


    if __name__ == '__main__':
     unittest.main(argv=['-v'], exit=False)  # Uncomment to run in notebook


## 13) Unit test with `assertRaises`

Write a `unittest.TestCase` verifying that adding a duplicate ISBN to a `Catalog` raises `DuplicateBookError`.
Use `setUp` to create a fresh `Catalog` and seed it with one book.

In [ ]:
# Your code here
import unittest

class TestCatalogDuplicates(unittest.TestCase):
    def setUp(self):
        self.catalog = Catalog()
        self.book = Book(isbn="1234567890", title="Test Book")
        self.catalog.add_book(self.book)
    def test_duplicate_raises(self):
         duplicate = Book(isbn="1234567890", title="Another Test Book")
         with self.assertRaises(DuplicateBookError):
            self.catalog.add_book(duplicate)

    if __name__ == '__main__':
     unittest.main(argv=['-v'], exit=False)

## 14) Subtests for multiple bad IDs (optional pattern)

Using `unittest`, write a single test method that loops over a collection of **invalid** member IDs and, using `self.subTest`, asserts that `sanitize_member_id` raises `ValueError` for each.

In [ ]:
# Your code here
import unittest

class TestMemberIdSanitization(unittest.TestCase):
    def test_bad_ids(self):
        bad_values = ["", "   ", "abc!", "id with space", "###"]
        for value in bad_values:
            with self.subTest(value=value):
                with self.assertRaises(ValueError):
                    sanitize_member_id(value)

# if __name__ == '__main__':
#     unittest.main(argv=['-v'], exit=False)

## 15) try/except/else logging

Write `try_index_book(catalog: Catalog, book: Book) -> bool` that
- Tries to `add_book`
- On exception, returns `False`
- Uses `else` to return `True` only when no exception occurred

In [ ]:
# Your code here
def try_index_book(catalog: Catalog, book: Book) -> bool:
    try:
        catalog.add_book(book)
        return True
    except DuplicateBookError:
        return False

## 16) Graceful KeyboardInterrupt

Create `interactive_copies_prompt()` that repeatedly prompts the user for copies (use `input()`), converts using `parse_copies`, and prints the value.
If the user hits `Ctrl+C`, catch `KeyboardInterrupt` and print `Bye!` before returning.

In [ ]:
# Your code here
def interactive_copies_prompt():
    try:
        while True:
            user_input = input("Enter number of copies (positive integer): ")
            try:
                copies = parse_copies(user_input)
                print(f"Valid number of copies entered: {copies}")
                return copies
            except ValueError as e:
                print(f"⚠️ {e}. Please try again.")
    except KeyboardInterrupt:
        print("\nBye!")

# Note: Be mindful running this in notebooks; it's acceptable to provide the function without calling it.

## 17) Catalog validator that accumulates errors

Implement `validate_catalog_items(items: list[dict]) -> list[str]` that walks a list of book dicts and **collects** error messages rather than raising immediately.
Return a list of error strings (empty if valid).

In [ ]:
# Your code here
def validate_catalog_items(items):
    for item in items:
        isbn = item.get("isbn", "")
        try:
            validate_isbn(isbn)
        except ValueError as e:
            print(f"⚠️ Invalid ISBN '{isbn}': {e}")

## 18) Replace broad except

Refactor a function that currently uses a bare `except:` to instead catch **only** `ValueError` and `TypeError` and re-raise unknown exceptions. Provide a before/after example and explain why broad except is risky.

In [ ]:
# Your code here
def before_style(x):
    try:
        return int(x)
    except:
        return 0  # BAD: swallows everything

def after_style(x):
    try:
        return int(x)
    except (ValueError, TypeError):
        return 0  
    except Exception as e:
        raise e

# Add a short explanation in comments.
# Broad except clauses (like in before_style) can hide unexpected errors and make debugging difficult.
# The after_style function improves this by only catching specific exceptions (ValueError, TypeError)

## 19) Exit codes on failure

Write `main(argv)` that attempts to load a catalog JSON path from `argv[1]` using `safe_load_catalog`.
- Return `0` on success, `2` on `FileNotFoundError`, `3` on `json.JSONDecodeError`, otherwise `1`.
Use `sys.exit(main(sys.argv))` pattern in a protected `if __name__ == '__main__':` block (comment it out in notebook).

In [ ]:
# Your code here
import sys

def main(argv: list[str]) -> int:
    if len(argv) < 2:
        print("Usage: main <catalog.json>")
        return 1

    path = argv[1]
    try:
        safe_load_catalog(path)
        return 0
    except FileNotFoundError:
        return 2
    except json.JSONDecodeError:
        return 3
    except Exception as e:
        # re-raise could be considered, but returning 1 per instructions
        return 1

# if __name__ == '__main__':
#     sys.exit(main(sys.argv))

## 20) End-to-end happy path test

Using `unittest`, write a test that:
- Builds a fresh `Catalog`
- Adds a valid `Book`
- Creates a `Loan` due tomorrow and asserts `assert_not_overdue` returns `True`
This is a **basic** end-to-end happy path—no Week 8+ features.

In [ ]:
# Your code here
import unittest
from datetime import datetime, timedelta

class TestHappyPath(unittest.TestCase):
    def test_happy_flow(self):
        catalog = Catalog()
        book = Book(title="The Great Gatsby", author="F. Scott Fitzgerald", copies=3)
        catalog.add_book(book)
        due_date = datetime.now() + timedelta(days=1)
        loan = Loan(book, due_date)

        self.assertTrue(loan.assert_not_overdue())

# if __name__ == '__main__':
#     unittest.main(argv=['-v'], exit=False)

## Python skills you'll need (Weeks 1–7)

- **Core syntax & data types:** variables, strings, ints/floats, booleans
- **Collections:** lists, dicts (basic use only)
- **Control flow:** `if/elif/else`, `for` loops, `while` loops
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O:** open/read/write text and JSON (basic)
- **Classes & objects:** defining simple classes, `__init__`, instance methods, attributes
- **Encapsulation basics:** private attributes (naming convention), validation via methods/properties
- **Methods:** instance/class/static methods (as introduced in Week 6)
- **Error handling:** `try` / `except` / `else` / `finally`, `raise`, custom exceptions (Week 7)
- **Basic testing:** `unittest.TestCase`, `assertRaises`, `setUp`/`tearDown`, `subTest` (Week 7)
- **Standard library familiarity:** `datetime`, `json`, `sys`, built-in exceptions
